Convert the best Keras model → TFLite
with a **true “none”** output when no target sound is present.

In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# ==================== PATHS ====================
MODELS_DIR       = '../models/models_approach1'
RESULTS_DIR      = '../results/results_approach1'
SAVED_MODELS_DIR = os.path.join(MODELS_DIR, 'saved_models')
TFLITE_DIR       = '../models/tflite_models'

os.makedirs(TFLITE_DIR, exist_ok=True)

# ==================== CUSTOM LAYERS (REQUIRED FOR LOADING) ====================
# These must match exactly what was used to save the model
@tf.keras.utils.register_keras_serializable()
class YamnetEmbedding(tf.keras.layers.Layer):
    def __init__(self, yamnet_model, **kwargs):
        super().__init__(**kwargs)
        self.yamnet_model = yamnet_model

    def call(self, inputs):
        def map_fn(wave):
            scores, embeddings, _ = self.yamnet_model(wave)
            return tf.reduce_mean(embeddings, axis=0)
        return tf.map_fn(map_fn, inputs, dtype=tf.float32)

@tf.keras.utils.register_keras_serializable()
class ScalerLayer(tf.keras.layers.Layer):
    def __init__(self, mean=None, scale=None, **kwargs):
        super(ScalerLayer, self).__init__(**kwargs)
        if mean is not None:
            self.mean_weight = tf.Variable(initial_value=mean, trainable=False, dtype=tf.float32)
        if scale is not None:
            self.scale_weight = tf.Variable(initial_value=scale, trainable=False, dtype=tf.float32)

    def call(self, inputs):
        return (inputs - self.mean_weight) / self.scale_weight

    def get_config(self):
        return super(ScalerLayer, self).get_config()

@tf.keras.utils.register_keras_serializable()
class TemperatureSoftmax(tf.keras.layers.Layer):
    def __init__(self, temperature=1.0, **kwargs):
        super(TemperatureSoftmax, self).__init__(**kwargs)
        self.temperature = float(temperature)

    def call(self, inputs):
        return tf.nn.softmax(inputs / self.temperature)

    def get_config(self):
        config = super(TemperatureSoftmax, self).get_config()
        config.update({"temperature": self.temperature})
        return config

print("Environment set up and Custom Layers defined.")

c:\Users\301364822\AppData\Local\miniconda3\envs\AI-Capstone-FINAL\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version





Environment set up and Custom Layers defined.


In [2]:
def convert_to_tflite(model_name):
    # 1. Construct path to the SavedModel directory
    # (Matches how we saved it: 'custom_keras_nn_full')
    saved_model_path = os.path.join(SAVED_MODELS_DIR, f'{model_name.lower()}_full')
    
    print(f"\nLoading SavedModel from: {saved_model_path}")
    
    # Load the model to ensure it works
    try:
        model = tf.keras.models.load_model(
            saved_model_path,
            custom_objects={
                'YamnetEmbedding': YamnetEmbedding,
                'ScalerLayer': ScalerLayer,
                'TemperatureSoftmax': TemperatureSoftmax
            }
        )
        print("  Model loaded successfully. Starting conversion...")
    except Exception as e:
        print(f"  Error loading model: {e}")
        return

    # ==================== CONVERTER SETUP ====================
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    # CRITICAL: Allow TensorFlow Ops (Flex Delegate)
    # YAMNet uses audio signal processing ops (STFT) that are not in standard TFLite.
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS, # Standard TFLite ops
        tf.lite.OpsSet.SELECT_TF_OPS    # Fallback to TF for complex audio ops
    ]
    
    # Optimization 1: Standard Float32
    print("  Converting (Float32)...")
    tflite_model = converter.convert()
    
    # Save Float32
    float_path = os.path.join(TFLITE_DIR, f'{model_name}_float32.tflite')
    with open(float_path, 'wb') as f:
        f.write(tflite_model)
    print(f"  Saved: {float_path} ({len(tflite_model)/1024/1024:.2f} MB)")

    # Optimization 2: Dynamic Range Quantization (OPTIONAL but Recommended)
    # This reduces weights to int8, making the model 4x smaller
    print("  Converting (Dynamic Range Quantization)...")
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_quant_model = converter.convert()
    
    # Save Quantized
    quant_path = os.path.join(TFLITE_DIR, f'{model_name}_quant.tflite')
    with open(quant_path, 'wb') as f:
        f.write(tflite_quant_model)
    print(f"  Saved: {quant_path} ({len(tflite_quant_model)/1024/1024:.2f} MB)")

# Run conversion for the winning model
convert_to_tflite('Custom_Keras_NN')


Loading SavedModel from: ../models/models_approach1\saved_models\custom_keras_nn_full



  Model loaded successfully. Starting conversion...
  Converting (Float32)...
INFO:tensorflow:Assets written to: C:\Users\301364~1\AppData\Local\Temp\tmp88bug_mb\assets


INFO:tensorflow:Assets written to: C:\Users\301364~1\AppData\Local\Temp\tmp88bug_mb\assets


  Saved: ../models/tflite_models\Custom_Keras_NN_float32.tflite (13.36 MB)
  Converting (Dynamic Range Quantization)...
INFO:tensorflow:Assets written to: C:\Users\301364~1\AppData\Local\Temp\tmpj9dkt6pc\assets


INFO:tensorflow:Assets written to: C:\Users\301364~1\AppData\Local\Temp\tmpj9dkt6pc\assets


  Saved: ../models/tflite_models\Custom_Keras_NN_quant.tflite (3.54 MB)


In [3]:
def verify_tflite(tflite_path):
    print(f"\nVerifying: {tflite_path}")
    try:
        # Load Interpreter
        interpreter = tf.lite.Interpreter(model_path=tflite_path)
        interpreter.allocate_tensors()

        # Get details
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Create dummy input (15360 samples of audio)
        input_shape = input_details[0]['shape']
        # If shape is dynamic [1, 15360], numpy needs exact match
        dummy_input = np.random.randn(*input_shape).astype(np.float32)

        # Run Inference
        interpreter.set_tensor(input_details[0]['index'], dummy_input)
        interpreter.invoke()
        
        # Get Output
        output_data = interpreter.get_tensor(output_details[0]['index'])
        print(f"  Success! Output shape: {output_data.shape}")
        print(f"  Output Probabilities Sum: {np.sum(output_data):.4f}")
        
    except Exception as e:
        print(f"  Verification Failed: {e}")

# Verify both models
verify_tflite(os.path.join(TFLITE_DIR, 'Custom_Keras_NN_float32.tflite'))
verify_tflite(os.path.join(TFLITE_DIR, 'Custom_Keras_NN_quant.tflite'))


Verifying: ../models/tflite_models\Custom_Keras_NN_float32.tflite
  Success! Output shape: (1, 6)
  Output Probabilities Sum: 1.0000

Verifying: ../models/tflite_models\Custom_Keras_NN_quant.tflite
  Success! Output shape: (1, 6)
  Output Probabilities Sum: 1.0000


In [6]:
import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report
from glob import glob
import joblib
import pandas as pd

# ==================== 1. LOAD TEST DATA (RAW AUDIO) ====================
print("\n" + "="*80)
print("PREPARING TEST DATA FOR TFLITE EVALUATION")
print("="*80)

# Load Label Encoder
label_encoder = joblib.load(os.path.join(MODELS_DIR, 'label_encoder.pkl'))
classes = label_encoder.classes_

# Map folder names to class names
category_mapping = {
    'alarm_clock': 'Alarm_Clock',
    'car_horn': 'Car_Horn',
    'glass_breaking': 'Glass_Breaking',
    'gunshot': 'Gunshot',
    'siren': 'Siren'
}

raw_audio_list = []
labels_list = []
target_length = 15360  # 0.96s @ 16kHz

print("Loading .npy files...")
for folder_name, class_name in category_mapping.items():
    folder_path = os.path.join('../data/split_processed/test', folder_name) # Hardcoded path based on previous context
    if not os.path.exists(folder_path): continue
    
    npy_files = glob(os.path.join(folder_path, '*.npy'))
    for npy_file in npy_files:
        try:
            audio = np.load(npy_file)
            if audio.ndim > 1: audio = audio.flatten()
            if len(audio) == target_length:
                raw_audio_list.append(audio)
                labels_list.append(class_name)
        except: pass

X_test_tflite = np.array(raw_audio_list, dtype=np.float32)
y_test_labels = np.array(labels_list)
y_test_encoded = label_encoder.transform(y_test_labels)

print(f"Loaded {len(X_test_tflite)} samples for testing.")

# ==================== 2. TFLITE INFERENCE FUNCTION ====================
def evaluate_tflite(model_path, X_data, y_true):
    print(f"\nEvaluating TFLite Model: {os.path.basename(model_path)}")
    
    # Load Interpreter
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    input_index = input_details[0]['index']
    output_index = output_details[0]['index']
    
    predictions = []
    start_time = time.time()
    
    # TFLite usually handles one sample at a time in Python
    # (Unless resized, but loop is safer for verification)
    for i in tqdm(range(len(X_data)), desc="Running Inference"):
        # Get sample and ensure shape is (1, 15360)
        sample = X_data[i].reshape(1, -1)
        
        interpreter.set_tensor(input_index, sample)
        interpreter.invoke()
        
        output = interpreter.get_tensor(output_index)
        pred_label = np.argmax(output)
        predictions.append(pred_label)

    total_time = (time.time() - start_time) * 1000 # to ms
    avg_latency = total_time / len(X_data)
    
    # Metrics
    acc = accuracy_score(y_true, predictions)
    f1 = f1_score(y_true, predictions, average='macro')
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    
    print(f"  Accuracy:      {acc:.4f}")
    print(f"  F1 Macro:      {f1:.4f}")
    print(f"  Avg Latency:   {avg_latency:.2f} ms/sample")
    print(f"  Model Size:    {model_size:.2f} MB")
    
    return {
        'Model': os.path.basename(model_path),
        'Accuracy': acc,
        'F1': f1,
        'Latency': avg_latency,
        'Size_MB': model_size
    }

# ==================== 3. RUN EVALUATION ====================
tflite_results = []

# 1. Test Float32 Model
float_model_path = os.path.join(TFLITE_DIR, 'Custom_Keras_NN_float32.tflite')
if os.path.exists(float_model_path):
    res = evaluate_tflite(float_model_path, X_test_tflite, y_test_encoded)
    tflite_results.append(res)

# 2. Test Quantized Model
quant_model_path = os.path.join(TFLITE_DIR, 'Custom_Keras_NN_quant.tflite')
if os.path.exists(quant_model_path):
    res = evaluate_tflite(quant_model_path, X_test_tflite, y_test_encoded)
    tflite_results.append(res)

# ==================== 4. SUMMARY TABLE ====================
if tflite_results:
    print("\n" + "="*60)
    print("TFLITE PERFORMANCE SUMMARY")
    print("="*60)
    df_res = pd.DataFrame(tflite_results)
    print(df_res[['Model', 'Accuracy', 'F1', 'Latency', 'Size_MB']].to_string(index=False))
    
    # Optional: Save results
    df_res.to_csv(os.path.join(RESULTS_DIR, 'tflite_evaluation_results.csv'), index=False)


PREPARING TEST DATA FOR TFLITE EVALUATION
Loading .npy files...
Loaded 1460 samples for testing.

Evaluating TFLite Model: Custom_Keras_NN_float32.tflite


Running Inference: 100%|██████████| 1460/1460 [00:28<00:00, 50.79it/s]


  Accuracy:      0.8322
  F1 Macro:      0.7162
  Avg Latency:   19.69 ms/sample
  Model Size:    13.36 MB

Evaluating TFLite Model: Custom_Keras_NN_quant.tflite


Running Inference: 100%|██████████| 1460/1460 [00:23<00:00, 60.90it/s]

  Accuracy:      0.0178
  F1 Macro:      0.0250
  Avg Latency:   16.42 ms/sample
  Model Size:    3.54 MB

TFLITE PERFORMANCE SUMMARY
                         Model  Accuracy       F1   Latency   Size_MB
Custom_Keras_NN_float32.tflite  0.832192 0.716225 19.692324 13.362617
  Custom_Keras_NN_quant.tflite  0.017808 0.024989 16.420441  3.535515
